In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
def ts(x):
    print(type(x))
    print(np.shape(x))

In [3]:
fmly_plans = pd.read_csv(r'prepped_fmly.txt', sep = '\t')
indv_plans = pd.read_csv(r'prepped_indv.txt', sep = '\t')

In [4]:
x_columns = ['PlanType','DentalOnlyPlan','MarketCoverage','MetalLevel','OutOfCountryCoverage',
             'StateCode','OutOfServiceAreaCoverage','Tobacco','Age']
fmly_plans_x = fmly_plans[x_columns]
indv_plans_x = indv_plans[x_columns]

In [5]:
Couple_y  = fmly_plans["Couple"]
PrimarySubscriberAndOneDependent_y  = fmly_plans["PrimarySubscriberAndOneDependent"]
PrimarySubscriberAndTwoDependents_y  = fmly_plans["PrimarySubscriberAndTwoDependents"]
PrimarySubscriberAndThreeOrMoreDependents_y  = fmly_plans["PrimarySubscriberAndThreeOrMoreDependents"]
CoupleAndOneDependent_y  = fmly_plans["CoupleAndOneDependent"]
CoupleAndTwoDependents_y  = fmly_plans["CoupleAndTwoDependents"]
CoupleAndThreeOrMoreDependents_y  = fmly_plans["CoupleAndThreeOrMoreDependents"]
IndividualRate_y = indv_plans["IndividualRate"]

In [6]:
from sklearn import preprocessing

In [7]:
label_encoder_lst = []
encoded_fmly_data = fmly_plans_x.copy()
encoded_indv_data = indv_plans_x.copy()

for col in x_columns:
    le = preprocessing.LabelEncoder()
    le.fit(fmly_plans_x[col].append(indv_plans_x[col]))
    label_encoder_lst.append(le)
    encoded_fmly_data[col] = le.transform(fmly_plans_x[col])
    encoded_indv_data[col] = le.transform(indv_plans_x[col])

In [8]:
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [9]:
def train_test_model(X, y, yNname):
    reg = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    reg.fit(X_train, y_train)
    '''
    print("----------------------------------------")
    print("Rate for {}".format(yNname))
    print("Training MSE: {}".format(mean_squared_error(y_train, reg.predict(X_train))))
    print("Testing  MSE: {}".format(mean_squared_error(y_test, reg.predict(X_test))))
    '''
    joblib.dump(reg, yNname + '.pkl')

In [10]:
import os
train_ind_exists = os.path.isfile('yes_train_the_model.txt')
if train_ind_exists:
    train_test_model(encoded_fmly_data, Couple_y, 'Couple')
    train_test_model(encoded_fmly_data, PrimarySubscriberAndOneDependent_y, 'PrimarySubscriberAndOneDependent')
    train_test_model(encoded_fmly_data, PrimarySubscriberAndTwoDependents_y, 'PrimarySubscriberAndTwoDependents')
    train_test_model(encoded_fmly_data, PrimarySubscriberAndThreeOrMoreDependents_y, 'PrimarySubscriberAndThreeOrMoreDependents')
    train_test_model(encoded_fmly_data, CoupleAndOneDependent_y, 'CoupleAndOneDependent')
    train_test_model(encoded_fmly_data, CoupleAndTwoDependents_y, 'CoupleAndTwoDependents')
    train_test_model(encoded_fmly_data, CoupleAndThreeOrMoreDependents_y, 'CoupleAndThreeOrMoreDependents')
    train_test_model(encoded_indv_data, IndividualRate_y, 'IndividualRate')

In [11]:
y_lst = ['Couple', 'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents', 
         'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent', 
         'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents']
y_lst_desc = ['Couple', 'Primary Subscriber And One Dependent', 'Primary Subscriber And Two Dependents',
              'Primary Subscriber And Three Or More Dependents', 'Couple And One Dependent', 
              'Couple And Two Dependents', 'Couple And Three Or More Dependents']

In [12]:
def get_quote(X, yNname):
    reg = joblib.load(yNname+'.pkl')
    return reg.predict(X)

In [13]:
ip_PlanType = 'PPO'
ip_DentalOnlyPlan = 'Yes'
ip_MarketCoverage = 'Individual'
ip_MetalLevel = 'High'
ip_OutOfCountryCoverage = 'No'
ip_StateCode = 'NJ'
ip_OutOfServiceAreaCoverage = 'Yes'
ip_Tobacco = 'Tobacco-User'
ip_Age = '30'

In [14]:
input_x = pd.DataFrame([[ip_PlanType, ip_DentalOnlyPlan, ip_MarketCoverage, ip_MetalLevel, ip_OutOfCountryCoverage, 
                         ip_StateCode, ip_OutOfServiceAreaCoverage, ip_Tobacco, ip_Age]])
input_x_encoded = input_x.copy()
for i in input_x.columns:
    input_x_encoded[i] = label_encoder_lst[i].transform(input_x[i])

In [15]:
for i in range(len(y_lst)):
    print("Quote for {:47} - ${:.2f}".format(y_lst_desc[i], get_quote(input_x_encoded,y_lst[i])[0]  ))

Quote for Couple                                          - $70.23
Quote for Primary Subscriber And One Dependent            - $71.76
Quote for Primary Subscriber And Two Dependents           - $109.47
Quote for Primary Subscriber And Three Or More Dependents - $144.95
Quote for Couple And One Dependent                        - $113.30
Quote for Couple And Two Dependents                       - $135.90
Quote for Couple And Three Or More Dependents             - $174.98


In [16]:
for col in x_columns:
    print("--------------------------------------------")
    print(col)
    print(fmly_plans_x[col].append(indv_plans_x[col]).unique().tolist())

--------------------------------------------
PlanType
['PPO', 'EPO', 'POS', 'HMO', 'Indemnity']
--------------------------------------------
DentalOnlyPlan
['Yes', 'No']
--------------------------------------------
MarketCoverage
['Individual', 'SHOP (Small Group)']
--------------------------------------------
MetalLevel
['High', 'Low', 'Gold', 'Silver', 'Bronze', 'Platinum', 'Catastrophic']
--------------------------------------------
OutOfCountryCoverage
['No', 'Yes']
--------------------------------------------
StateCode
['NJ', 'PA', 'NH', 'OH', 'VA', 'AZ', 'IN', 'LA', 'NC', 'TN', 'TX', 'DE', 'IA', 'IL', 'MI', 'NE', 'SD', 'AL', 'AK', 'AR', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'HI', 'ID', 'KS', 'KY', 'ME', 'MD', 'MA', 'MN', 'MS', 'MO', 'MT', 'NV', 'NM', 'NY', 'ND', 'OK', 'OR', 'RI', 'SC', 'UT', 'VT', 'WA', 'WV', 'WI', 'WY']
--------------------------------------------
OutOfServiceAreaCoverage
['Yes', 'No']
--------------------------------------------
Tobacco
['Tobacco-User', 'Non-Toba

In [17]:
fmly_plans

PlanType DentalOnlyPlan      MarketCoverage MetalLevel  \
0        PPO            Yes          Individual       High   
1        PPO            Yes          Individual        Low   
2        PPO            Yes          Individual       High   
3        PPO            Yes          Individual       High   
4        PPO            Yes          Individual       High   
5        PPO            Yes          Individual       High   
6        PPO            Yes          Individual       High   
7        PPO            Yes          Individual       High   
8        PPO            Yes          Individual       High   
9        PPO            Yes          Individual       High   
10       PPO            Yes          Individual       High   
11       PPO            Yes          Individual        Low   
12       PPO            Yes          Individual        Low   
13       PPO            Yes          Individual        Low   
14       PPO            Yes          Individual        Low   
15       PPO            Yes          Individual        Low   
16       PPO            Yes          Individual        Low   
17       PPO            Yes          Individual        Low   
18       PPO            Yes          Individual        Low   
19       PPO            Yes          Individual        Low   
20       EPO            Yes          Individual        Low   
21       EPO            Yes          Individual        Low   
22       EPO            Yes          Individual        Low   
23       EPO            Yes          Individual       High   
24       EPO            Yes          Individual       High   
25       EPO            Yes          Individual       High   
26       EPO            Yes  SHOP (Small Group)        Low   
27       EPO            Yes  SHOP (Small Group)       High   
28       PPO            Yes  SHOP (Small Group)       High   
29       POS            Yes  SHOP (Small Group)       High   
..       ...            ...                 ...        ...   
488      PPO            Yes  SHOP (Small Group)        Low   
489      PPO            Yes  SHOP (Small Group)        Low   
490      PPO            Yes  SHOP (Small Group)        Low   
491      PPO            Yes  SHOP (Small Group)        Low   
492      PPO            Yes  SHOP (Small Group)        Low   
493      PPO            Yes  SHOP (Small Group)        Low   
494      PPO            Yes  SHOP (Small Group)        Low   
495      PPO            Yes  SHOP (Small Group)        Low   
496      PPO            Yes  SHOP (Small Group)        Low   
497      PPO            Yes  SHOP (Small Group)        Low   
498      PPO            Yes  SHOP (Small Group)        Low   
499      PPO            Yes  SHOP (Small Group)        Low   
500      PPO            Yes  SHOP (Small Group)        Low   
501      PPO            Yes  SHOP (Small Group)        Low   
502      PPO            Yes  SHOP (Small Group)        Low   
503      PPO            Yes  SHOP (Small Group)        Low   
504      PPO            Yes  SHOP (Small Group)        Low   
505      PPO            Yes  SHOP (Small Group)        Low   
506      PPO            Yes  SHOP (Small Group)        Low   
507      PPO            Yes  SHOP (Small Group)        Low   
508      PPO            Yes  SHOP (Small Group)        Low   
509      PPO            Yes  SHOP (Small Group)        Low   
510      PPO            Yes  SHOP (Small Group)        Low   
511      PPO            Yes  SHOP (Small Group)        Low   
512      PPO            Yes  SHOP (Small Group)        Low   
513      PPO            Yes  SHOP (Small Group)       High   
514      PPO            Yes  SHOP (Small Group)       High   
515      PPO            Yes  SHOP (Small Group)       High   
516      PPO            Yes  SHOP (Small Group)       High   
517      PPO            Yes  SHOP (Small Group)       High   

    OutOfCountryCoverage StateCode OutOfServiceAreaCoverage           Tobacco  \
0                     No        NJ                      Yes      Tobacco-User 